# Data


## Dataset

### import some packags

In [2]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

In [3]:
# 自定义数据集类，继承自PyTorch的Dataset类
class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        """
        初始化数据集
        Args:
            data_dir (str): 数据目录路径，包含metadata.json, mapping.json和特征文件
            segment_len (int, optional): 每个样本的mel频谱图长度. 默认为128帧.
        """
        self.data_dir = data_dir
        self.segment_len = segment_len

        # 加载说话人名称到ID的映射文件
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]  # 获取说话人到ID的映射字典

        # 加载训练数据的元数据
        metadata_path = Path(data_dir) / "metadata.json"
        # 元数据结构为: {"speakers": {speaker1: [{feature_path: "...", ...}, ...], ...}}
        metadata = json.load(open(metadata_path))["speakers"]

        # 获取总说话人数
        self.speaker_num = len(metadata.keys())
        
        # 初始化数据列表，每个元素为[特征文件路径, 说话人ID]
        self.data = []
        # 遍历每个说话人的所有语音片段
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                # 将特征路径和对应的说话人ID加入数据列表
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        """返回数据集的总样本数"""
        return len(self.data)

    def __getitem__(self, index):
        """
        获取单个样本数据，这是Dataset类的核心方法
        Args:
            index (int): 样本索引
        Returns:
            tuple: (mel频谱图, 说话人ID)
        """
        # 获取特征文件路径和说话人ID
        feat_path, speaker = self.data[index]
        
        # 加载预处理的mel频谱图特征
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # 将mel频谱图切割成固定长度的片段
        if len(mel) > self.segment_len:
            # 随机获取片段的起始点，确保不越界
            start = random.randint(0, len(mel) - self.segment_len)
            # 截取指定长度的mel片段并转换为FloatTensor
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            # 如果mel长度不足，直接使用全部内容
            mel = torch.FloatTensor(mel)
        
        # 将说话人ID转换为LongTensor，用于后续的损失计算
        speaker = torch.FloatTensor([speaker]).long()
        
        return mel, speaker

    def get_speaker_number(self):
        """返回数据集中说话人的总数"""
        return self.speaker_num
        

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
def collate_batch(batch):
    """
    自定义批处理函数，用于处理变长序列的填充
    Args:
        batch: 一个批次的样本列表，每个样本为 (mel, speaker)
    Returns:
        tuple: 填充后的mel频谱和说话人标签
    """
    # 将batch中的mel和speaker分别解压缩到两个元组中
    # mel: 包含多个不同长度的mel频谱图的列表
    # speaker: 包含对应说话人ID的列表
    mel, speaker = zip(*batch)
    
    # 对mel频谱进行填充，使一个batch内的所有序列长度相同
    # batch_first=True: 输出的张量形状为 (batch_size, seq_len, features)
    # padding_value=-20: 填充值为-20，对应log10(10^-20)，是一个很小的数值（近似于静音帧）
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)
    # 输出mel形状: (batch_size, max_length, n_mels=40)
    
    # 将说话人ID列表转换为LongTensor并返回
    return mel, torch.FloatTensor(speaker).long()

def get_dataloader(data_dir, batch_size, n_workers):
    """
    生成训练和验证数据加载器
    Args:
        data_dir (str): 数据目录路径
        batch_size (int): 每个批次的样本数量
        n_workers (int): 数据加载时使用的子进程数
    Returns:
        tuple: (训练数据加载器, 验证数据加载器, 说话人总数)
    """
    # 创建自定义数据集实例
    dataset = myDataset(data_dir)
    # 获取数据集中说话人的总数（用于模型输出层的设计）
    speaker_num = dataset.get_speaker_number()
    
    # 将数据集按9:1的比例分割为训练集和验证集
    trainlen = int(0.9 * len(dataset))  # 90% 用于训练
    lengths = [trainlen, len(dataset) - trainlen]  # 训练集和验证集的大小
    trainset, validset = random_split(dataset, lengths)  # 随机分割数据集

    # 创建训练数据加载器
    train_loader = DataLoader(
        trainset,           # 训练数据集
        batch_size=batch_size,  # 批次大小
        shuffle=True,       # 每个epoch打乱数据顺序，防止模型记忆顺序
        drop_last=True,     # 丢弃最后一个不完整的批次，保证批次大小一致
        num_workers=n_workers,  # 使用多进程加载数据，加速数据预处理
        pin_memory=True,    # 将数据锁页内存，加速GPU数据传输
        collate_fn=collate_batch,  # 使用自定义的批处理函数
    )
    
    # 创建验证数据加载器
    valid_loader = DataLoader(
        validset,           # 验证数据集
        batch_size=batch_size,  # 批次大小
        num_workers=n_workers,  # 使用多进程加载数据
        drop_last=True,     # 丢弃最后一个不完整的批次
        pin_memory=True,    # 将数据锁页内存
        collate_fn=collate_batch,  # 使用相同的批处理函数
        # 注意：验证集不需要shuffle，以便更好地评估模型性能
    )

    return train_loader, valid_loader, speaker_num

# Model
TransformerEncoderLayer（Transformer编码器层）：

基于Attention Is All You Need论文中的基础Transformer编码器层

参数：

d_model：输入特征的预期数量（必需）

nhead：多头注意力模型中的头数（必需）

dim_feedforward：前馈网络模型的维度（默认=2048）

dropout：dropout值（默认=0.1）

activation：中间层的激活函数，relu或gelu（默认=relu）

TransformerEncoder（Transformer编码器）：

Transformer编码器是由N个Transformer编码器层堆叠而成的

参数：

encoder_layer：TransformerEncoderLayer()类的实例（必需）

num_layers：编码器中子编码器层的数量（必需）

norm：层归一化组件（可选）

In [ ]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.1):
    super().__init__()
    # 将输入特征的维度从40投影到d_model
    self.prenet = nn.Linear(40, d_model)
    
    # TODO:
    #   将Transformer改为Conformer
    #   https://arxiv.org/abs/2005.08100
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model,        # 模型维度
      dim_feedforward=256,    # 前馈网络维度
      nhead=2                 # 多头注意力头数
    )
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

    # 将特征维度从d_model投影到说话人数量
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, d_model),  # 线性层
      nn.ReLU(),                    # ReLU激活函数
      nn.Linear(d_model, n_spks),   # 输出层（说话人分类）
    )

  def forward(self, mels):
    """
    前向传播函数
    参数:
      mels: (批次大小, 序列长度, 40) - 梅尔频谱特征
    返回:
      out: (批次大小, n_spks) - 说话人分类概率
    """
    # out: (批次大小, 序列长度, d_model)
    out = self.prenet(mels)  # 特征维度投影
    
    # out: (序列长度, 批次大小, d_model)
    out = out.permute(1, 0, 2)  # 调整维度顺序以适应Transformer输入要求
    
    # 编码器层期望特征形状为 (序列长度, 批次大小, d_model)
    out = self.encoder_layer(out)  # Transformer编码处理
    
    # out: (批次大小, 序列长度, d_model)
    out = out.transpose(0, 1)  # 恢复维度顺序
    
    # 均值池化：沿序列长度维度取平均
    stats = out.mean(dim=1)

    # out: (批次大小, n_spks) - 说话人分类结果
    out = self.pred_layer(stats)
    return out


# 学习率调度策略
对于Transformer架构，学习率调度方案的设计与卷积神经网络（CNN）有所不同。

现有研究表明，学习率预热机制能有效提升基于Transformer架构模型的训练效果。

预热调度方案具体表现为：

训练初始阶段将学习率设置为0

在预热周期内，学习率从0开始线性增长至预设的初始学习率值

In [ ]:
import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR
import math  # 需要导入math模块

#为什么这里有的形参后是：确定实参的数据类型
def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,           # 必须是Optimizer类型
    num_warmup_steps: int,          # 必须是整数
    num_training_steps: int,        # 必须是整数  
    num_cycles: float = 0.5,        # 浮点数，默认值0.5
    last_epoch: int = -1,           # 整数，默认值-1
):
    """
    创建带有预热期的余弦学习率调度器。
    
    该调度器首先在预热期内将学习率从0线性增加到优化器中设置的初始学习率，
    然后按照余弦函数的值从初始学习率递减到0。

    参数:
        optimizer (:class:`~torch.optim.Optimizer`):
            需要调度学习率的优化器
        num_warmup_steps (:obj:`int`):
            预热阶段的步数
        num_training_steps (:obj:`int`):
            总训练步数
        num_cycles (:obj:`float`, `可选`, 默认为 0.5):
            余弦调度中的周期数（默认值0.5表示遵循半个余弦波从最大值下降到0）
        last_epoch (:obj:`int`, `可选`, 默认为 -1):
            恢复训练时上一个epoch的索引

    返回:
        :obj:`torch.optim.lr_scheduler.LambdaLR`: 带有相应调度规则的学习率调度器
    """

    def lr_lambda(current_step):
        # 预热阶段
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # 衰减阶段
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(
            0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)